In [152]:
import physionet_challenge_utility_script as pc

import ecg_plot
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_addons as tfa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.preprocessing.sequence import pad_sequences

print(1)

%load_ext autoreload
%autoreload
%reload_ext autoreload


1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [153]:
DATADIR = "./SM22_MLData"

In [154]:
firstModelData = {}
firstModelData["gender"], firstModelData["age"], firstModelData["labels"], firstModelData["ecg_filenames"] = pc.import_key_data(DATADIR)
firstModelData["ecg_filenames"] = np.asarray(firstModelData["ecg_filenames"])

In [155]:
cache = firstModelData["labels"]

In [156]:
"""
Remaps labels into class values (Integer Encoding)
val: totalRhythm
1 - SR
2 - NSR
3 - Both
"""

print(len(firstModelData["labels"]))
for i in range(len(firstModelData["labels"])):
    splitLabels = firstModelData["labels"][i].split(",")
    isSR, isNSR, totalRhythm = 0, 0, 0
    if len(splitLabels) > 1:
        isNSR = 2 #NSR
    if not("426783006" in splitLabels):
        isNSR = 2 #NSR
    if "426783006" in splitLabels:
        isSR = 1 #SR
    totalRhythm = isSR + isNSR
    firstModelData["labels"][i] = str(totalRhythm)


43101


In [ ]:
firstModelData["labels"][0] = "2,undefined"
firstModelData["labels"]

In [168]:
firstModelData["age"], firstModelData["gender"] = pc.import_gender_and_age(firstModelData["age"], firstModelData["gender"])

gender data shape: 43101
age data shape: 43101


/Users/pranav/Documents/GitHub/ArrythmiaScienceMontgomery/physionet_challenge_utility_script.py:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  gender[np.where(gender == "Male")] = 0
/Users/pranav/Documents/GitHub/ArrythmiaScienceMontgomery/physionet_challenge_utility_script.py:42: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  gender[np.where(gender == "male")] = 0
/Users/pranav/Documents/GitHub/ArrythmiaScienceMontgomery/physionet_challenge_utility_script.py:43: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  gender[np.where(gender == "M")] = 0
/Users/pranav/Documents/GitHub/ArrythmiaScienceMontgomery/physionet_challenge_utility_script.py:44: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform

In [169]:
SNOMED_scored=pd.read_csv(f"{DATADIR}/SNOMEDMappings/SNOMED_mappings_scored.csv", sep=";")
SNOMED_unscored=pd.read_csv(f"{DATADIR}/SNOMEDMappings/FIRSTMODELMapping.csv", sep=",")

In [170]:
df_labels = pd.DataFrame(firstModelData["labels"])

In [173]:
y , snomed_classes = pc.onehot_encode(df_labels)

The classes we will look at are encoded as SNOMED CT codes:
['1' '2' '3' 'undefined']
classes: 3


In [174]:
y

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [175]:
y_all_comb = pc.get_labels_for_all_combinations(y)

In [176]:
folds = pc.split_data(firstModelData["labels"], y_all_comb)

Training split: 38790
Validation split: 4311


In [177]:
order_array = folds[0][0]

In [178]:
def shuffle_batch_generator_demo(batch_size, gen_x,gen_y, gen_z): 
    np.random.shuffle(order_array)
    batch_features = np.zeros((batch_size,5000, 12))
    batch_labels = np.zeros((batch_size,snomed_classes.shape[0])) #drop undef class
    batch_demo_data = np.zeros((batch_size,2))
    while True:
        for i in range(batch_size):

            batch_features[i] = next(gen_x)
            batch_labels[i] = next(gen_y)
            batch_demo_data[i] = next(gen_z)

        X_combined = [batch_features, batch_demo_data]
        yield X_combined, batch_labels
        
def shuffle_batch_generator(batch_size, gen_x,gen_y): 
    np.random.shuffle(order_array)
    batch_features = np.zeros((batch_size,5000, 12))
    batch_labels = np.zeros((batch_size,snomed_classes.shape[0])) #drop undef class
    while True:
        for i in range(batch_size):

            batch_features[i] = next(gen_x)
            batch_labels[i] = next(gen_y)
            
        yield batch_features, batch_labels

def generate_y_shuffle(y_train):
    while True:
        for i in order_array:
            y_shuffled = y_train[i]
            yield y_shuffled


def generate_X_shuffle(X_train):
    while True:
        for i in order_array:
                #if filepath.endswith(".mat"):
                    data, header_data = pc.load_challenge_data(X_train[i])
                    X_train_new = pad_sequences(data, maxlen=5000, truncating='post',padding="post")
                    X_train_new = X_train_new.reshape(5000,12)
                    yield X_train_new

def generate_z_shuffle(age_train, gender_train):
    while True:
        for i in order_array:
            gen_age = age_train[i]
            gen_gender = gender_train[i]
            z_train = [gen_age , gen_gender]
            yield z_train

In [179]:
new_weights=pc.calculating_class_weights(y)

In [180]:
keys = np.arange(0,3,1)
weight_dictionary = dict(zip(keys, new_weights.T[1]))
weight_dictionary

{0: 2.34703768242213, 1: 0.9683441923163334, 2: 1.847608024691358}

In [181]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_AUC', factor=0.1, patience=1, verbose=1, mode='max',
    min_delta=0.0001, cooldown=0, min_lr=0
)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=2)

In [182]:
from scipy import optimize
def thr_chall_metrics(thr, label, output_prob):
    return -pc.compute_challenge_metric_for_opt(label, np.array(output_prob>thr))

In [192]:
def myFCN(classes=27):
    inputlayer = keras.layers.Input(shape=(5000,12)) 

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=8,input_shape=(5000,12), padding='same')(inputlayer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv1)

    outputlayer = keras.layers.Dense(classes, activation='sigmoid')(gap_layer)

    model = keras.Model(inputs=inputlayer, outputs=outputlayer)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
        name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,
    )])

    return model

In [193]:
model = pc.residual_network_1d(classes=3)

In [194]:
model.load_weights(f"{DATADIR}/bjornphysionet/resnet_model.h5")

ValueError: Cannot assign value to variable ' dense_13/kernel:0': Shape mismatch.The variable shape (128, 3), and the assigned value shape (128, 27) are incompatible.

In [195]:
print(shuffle_batch_generator(
    batch_size=batchsize, 
    gen_x=generate_X_shuffle(firstModelData["ecg_filenames"]), 
    gen_y=generate_y_shuffle(y)
))

<generator object shuffle_batch_generator at 0x28ee32270>


In [ ]:
batchsize = 30
model.fit(
    x=shuffle_batch_generator(batch_size=batchsize, gen_x=generate_X_shuffle(firstModelData["ecg_filenames"]), 
    gen_y=generate_y_shuffle(y)), 
    epochs=100, 
    steps_per_epoch=(len(order_array)/batchsize), 
    validation_data=pc.generate_validation_data(firstModelData["ecg_filenames"],y,folds[0][1]), 
    validation_freq=1, 
    class_weight=weight_dictionary, 
    callbacks=[reduce_lr,early_stop]
)

Epoch 1/100


2022-03-12 15:05:26.756005: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 610/1293 [=============>................] - ETA: 11:07 - loss: 0.5613 - accuracy: 0.8430 - Recall: 0.7387 - Precision: 0.7788 - AUC: 0.9067